# **Data Cleaning Notebook**

## Objectives

* Evaluate and handle missing data
* Clean data 

## Inputs

* outputs/dataset/collection/HospitalReadmissions.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\Andrias\\Desktop\\patient-readmission\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\Andrias\\Desktop\\patient-readmission'

# Load the dataset

In [4]:
import pandas as pd

data_path = 'outputs/datasets/collection/HospitalReadmissions.csv'

df = pd.read_csv(data_path)
df.head()

,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
0,[70-80),8,72,1,18,2,0,0,Missing,Circulatory,Respiratory,Other,no,no,no,yes,0
1,[70-80),3,34,2,13,0,0,0,Other,Other,Other,Other,no,no,no,yes,0
2,[50-60),5,45,0,18,0,0,0,Missing,Circulatory,Circulatory,Circulatory,no,no,yes,yes,1
3,[70-80),2,36,0,12,1,0,0,Missing,Circulatory,Other,Diabetes,no,no,yes,yes,1
4,[60-70),1,42,0,7,0,0,0,InternalMedicine,Other,Circulatory,Respiratory,no,no,no,yes,0


In [5]:
target_var = 'readmitted'

---

# Explore data

First we check for duplicated and missing values in order to decide what's the best way to handle those values.

### Duplicate Values

In [38]:
df.duplicated().sum()

0

 No duplicate values found

### Missing values

In [11]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                "PercentageOfDataset": missing_data_percentage,
                                "DataType": df.dtypes}
                                )
                        .sort_values(by=['PercentageOfDataset'], ascending=False)
                        .query("PercentageOfDataset > 0")
                        )

    return df_missing_data

In [12]:
EvaluateMissingData(df)

,RowsWithMissingData,PercentageOfDataset,DataType


When we run the above code cell, we don't get any missing values. However, as we noticed in our EDA, on the previous notebook, there some feature that contain the value 'Missing'. That's why we have adjust the function below to list any variables with the value "Missing".

In [13]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isin(['Missing']).sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                "PercentageOfDataset": missing_data_percentage,
                                "DataType": df.dtypes}
                                )
                        .sort_values(by=['PercentageOfDataset'], ascending=False)
                        .query("PercentageOfDataset > 0")
                        )

    return df_missing_data

In [14]:
EvaluateMissingData(df)

,RowsWithMissingData,PercentageOfDataset,DataType
medical_specialty,12382,49.53,object
diag_3,196,0.78,object
diag_2,42,0.17,object
diag_1,4,0.02,object


### Data cleaning Summary

As we can see above, the missing values have already been Imputed and replaced with value "Missing". Thus, ou only consideration is to drop the "medical-specialty" variable, since it doesn't contribute significantly to the prediction of patient's readmission and has almost 50% missing values.

The other 3 variables have insignificant amount of missing values, thus we can use them as they are.

---

# Split Train and Test set

In [15]:
from sklearn.model_selection import train_test_split

TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df[target_var],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

TrainSet shape: (20000, 17) 
TestSet shape: (5000, 17)


In [16]:
EvaluateMissingData(TrainSet)

,RowsWithMissingData,PercentageOfDataset,DataType
medical_specialty,9904,49.52,object
diag_3,155,0.78,object
diag_2,30,0.15,object
diag_1,4,0.02,object


In [17]:
EvaluateMissingData(TestSet)

,RowsWithMissingData,PercentageOfDataset,DataType
medical_specialty,2478,49.56,object
diag_3,41,0.82,object
diag_2,12,0.24,object


The next step is to drop the "medical_specialty" variable using the DropFeature transformer.

In [18]:
variables_to_drop = ['medical_specialty']
print(f"Variables to drop: {variables_to_drop}")

Variables to drop: ['medical_specialty']


In [19]:
from feature_engine.selection import DropFeatures

drop_features = DropFeatures(features_to_drop=variables_to_drop)
drop_features.fit(TrainSet)
drop_features_df = drop_features.transform(TrainSet)
drop_features_df.head()

,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
10263,[70-80),4,47,0,21,0,1,0,Other,Other,Other,no,no,yes,yes,1
18409,[60-70),3,50,1,11,0,0,0,Musculoskeletal,Circulatory,Diabetes,no,no,no,no,0
13047,[50-60),4,66,1,10,0,0,0,Other,Other,Diabetes,no,high,no,no,0
21371,[50-60),14,63,0,28,0,0,0,Respiratory,Circulatory,Diabetes,normal,no,no,yes,1
16392,[50-60),3,60,2,25,3,2,4,Other,Diabetes,Other,no,no,yes,yes,1


In our case the feature variable we dropped does not have an impact on the distribution of other variables since we removed a column.

Then below, we apply the transformation to our data.

In [20]:
TrainSet, TestSet = drop_features.transform(TrainSet) , drop_features.transform(TestSet)

In [21]:
EvaluateMissingData(TrainSet)

,RowsWithMissingData,PercentageOfDataset,DataType
diag_3,155,0.78,object
diag_2,30,0.15,object
diag_1,4,0.02,object


In [22]:
EvaluateMissingData(TestSet)

,RowsWithMissingData,PercentageOfDataset,DataType
diag_3,41,0.82,object
diag_2,12,0.24,object


As we can see, the transformer worker since the "medical_specialty" feature is gone and we have only the other 3 features, which contain insignificant amount of missing values and they already been transformed.

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)

## Train set

In [ ]:
TrainSet.to_csv('outputs/datasets/cleaned/TrainSetCleaned.csv', index=False)

## Test set

In [ ]:
TestSet.to_csv('outputs/datasets/cleaned/TestSetCleaned.csv', index=False)

## Next Steps

In the next notebook we will move to feature engineering.